In [1]:
import sqlalchemy
import pandas as pd

In [2]:
# add secrets here


In [3]:
DB_NAME_MAINNET = 'chainsync_qa_mainnet'
DB_NAME_PREPROD = 'chainsync_qa_preprod'
DB_NAME_PREVIEW = 'chainsync_qa_preview'

In [4]:
engine_mainnet = sqlalchemy.create_engine('postgresql://'+DB_USER+':'+DB_PASSWORD+'@'+DB_HOST+'/'+DB_NAME_MAINNET)
engine_preprod = sqlalchemy.create_engine('postgresql://'+DB_USER+':'+DB_PASSWORD+'@'+DB_HOST+'/'+DB_NAME_PREPROD)
engine_preview = sqlalchemy.create_engine('postgresql://'+DB_USER+':'+DB_PASSWORD+'@'+DB_HOST+'/'+DB_NAME_PREVIEW)

# analysis by staking

In [ ]:
df_mainnet = pd.read_sql('SELECT * FROM analysis.contracts_merged', engine_mainnet)


In [63]:
df_preprod = pd.read_sql('SELECT * FROM analysis.contracts_merged', engine_preprod)
df_preprod

,id,blockno,slotno,created,closed,status,size,ada_transacted,num_transactions,stakeaddress,is_runner
0,000a5348883fb3c0b96936cb7a51787024b9b71b31de80...,628191,20681601,1,1,closed,3126,326.605259,2,cce6bfa02342d29adce20d09d3824463bfb59b0753d7d1...,0
1,0047d28feec38404da50b375f8f35fd5fb83e6676db8da...,1424708,40139516,1,1,closed,1131,9830.774481,5,02ecf90aea5b27e1396e46eb6eb817d41bf4b70cfa23ce...,0
2,0047d28feec38404da50b375f8f35fd5fb83e6676db8da...,1424708,40139516,1,1,closed,1131,9830.774481,5,633871cc5286906147561cc3231220a1269ece7139081a...,0
3,0047d28feec38404da50b375f8f35fd5fb83e6676db8da...,1424708,40139516,1,1,closed,1131,9830.774481,5,fe7cc4fc92afd34cbd983ca1619745384b9f56c502cffe...,0
4,008f2fa2c40f6e033e38dca43039aebd14a8d4ac264947...,1456681,40925474,1,1,closed,5819,12884.970060,2,f5b3b821dcf83c502b8c14161de22f60c1f5115077bba0...,0
...,...,...,...,...,...,...,...,...,...,...,...
2719,ff67b2acfc779c6694231c2625887b4a2bd87118b7388d...,1574502,43876505,1,1,closed,541,1760.695656,6,ddf6ba798b55c603c80ca07bbc0f596a166a56d5abe1e9...,0
2720,ff67b2acfc779c6694231c2625887b4a2bd87118b7388d...,1574502,43876505,1,1,closed,541,1760.695656,6,bc96d47b25f5c68356666e63fcc366a5610f1df7c143a3...,0
2721,ff738f7c918ccb8c170445be021fbf6ca58bd4b341643b...,525398,18184937,1,1,closed,836,17.318216,4,9c43fbd9441d542647b478c858d08134b88e701dad92a5...,0
2722,ff738f7c918ccb8c170445be021fbf6ca58bd4b341643b...,525398,18184937,1,1,closed,836,17.318216,4,5d4ec93c82bb368db9750835df14c5cde19ebd7d0b2f02...,0


In [64]:
df_preview = pd.read_sql('SELECT * FROM analysis.contracts_merged', engine_preview)

In [65]:
def adjusted_slotno(val, offset):
    return int(val + offset)

def from_slots_to_date(df, offset):
    df['slotno_adjusted'] = df.apply(lambda row: adjusted_slotno(row['slotno'], offset),axis=1)    
    df = df.astype({'slotno_adjusted':'int'})
    df['date'] = pd.to_datetime(df['slotno_adjusted'], unit='s').dt.date
    df['month'] = pd.to_datetime(df['slotno_adjusted'], unit='s').dt.month
    df['year'] = pd.to_datetime(df['slotno_adjusted'], unit='s').dt.year
    return df

# df_mainnet = from_slots_to_date(df_mainnet, 1685297070 - 93730779)
df_preprod = from_slots_to_date(df_preprod, 1685297096 - 29613896)
df_preview = from_slots_to_date(df_preview, 1685297218 - 18641218)

In [66]:
# number of stake addresses created 1 or more contracts by month

def count_addresses_monthly(df):
    temp_year_month_stakeaddress_count = df.groupby(['year','month','stakeaddress'])['id'].size()
    temp_year_month_count = temp_year_month_stakeaddress_count.groupby(['year','month']).size()
    
    temp_year_month_count = temp_year_month_count.reset_index()
    temp_year_month_count.columns = [
        'year',
        'month',
        'num_stakeaddresses',
    ]

    return temp_year_month_count

# df_mainnet_monthly = count_addresses_monthly(df_mainnet)
df_preprod_monthly = count_addresses_monthly(df_preprod)
df_preview_monthly = count_addresses_monthly(df_preview)

In [67]:
def count_contracts(df):
    df_count = df.groupby(['year','month'])[['id']].count()
    df_count = df_count.reset_index()
    df_count.columns = [
        'year',
        'month',
        'num_contracts'
    ]
    return df_count

# df_mainnet_sum = get_sum(df_mainnet)
df_preprod_count = count_contracts(df_preprod)
df_preview_count = count_contracts(df_preview)

In [68]:
# get sum
def get_sum(df):
    df_sum = df.groupby(['year','month'])[['ada_transacted','num_transactions','is_runner']].sum()
    df_sum = df_sum.reset_index()
    df_sum.columns = [
        'year',
        'month',
        'ada_transacted_sum',
        'num_transactions_sum',
        'num_contracts_runner'
    ]
    return df_sum

# df_mainnet_sum = get_sum(df_mainnet)
df_preprod_sum = get_sum(df_preprod)
df_preview_sum = get_sum(df_preview)

In [69]:
# get mean
def get_mean(df):
    df_mean = df.groupby(['year','month'])[['ada_transacted','num_transactions']].mean()
    df_mean = df_mean.reset_index()
    df_mean.columns = [
        'year',
        'month',
        'ada_transacted_mean',
        'num_transactions_mean'
    ]
    return df_mean

# df_mainnet_mean = get_mean(df_mainnet)
df_preprod_mean = get_mean(df_preprod)
df_preview_mean = get_mean(df_preview)

In [70]:
# get runner
def get_runner(df):
    df_runner = df_preprod.groupby(['year','month','stakeaddress','is_runner'])['id'].size()
    df_runner = df_runner.reset_index()
    
    df_runner = df_runner[df_runner["is_runner"].isin([1])]
    
    df_runner = df_runner.groupby(['year','month']).size()
    df_runner = df_runner.reset_index()
    
    df_runner.columns = [
        'year',
        'month',
        'num_stakeaddresses_runner',        
    ]
    
    return df_runner

# # df_mainnet_runner = get_runner(df_mainnet)
df_preprod_runner = get_runner(df_preprod)
df_preview_runner = get_runner(df_preview)

In [71]:
def merge(df_monthly, df_count, df_runner, df_sum, df_mean):
    df_merged = pd.merge(df_monthly, df_count, how='left', left_on=['year','month'], right_on = ['year','month'])
    df_merged = pd.merge(df_merged, df_runner, how='left', left_on=['year','month'], right_on = ['year','month'])
    df_merged = pd.merge(df_merged, df_mean, how='left', left_on=['year','month'], right_on = ['year','month'])
    df_merged = pd.merge(df_merged, df_sum, how='left', left_on=['year','month'], right_on = ['year','month'])
    return df_merged

# df_mainnet_merge = merge(df_mainnet_monthly, df_mainnet_count, df_mainnet_runner, df_mainnet_sum, df_mainnet_mean)
df_preprod_merge = merge(df_preprod_monthly, df_preprod_count, df_preprod_runner, df_preprod_sum, df_preprod_mean)
df_preview_merge = merge(df_preview_monthly, df_preview_count, df_preview_runner, df_preview_sum, df_preview_mean)

In [72]:
# df_mainnet_merge['network'] = 'mainnet'
df_preprod_merge['network'] = 'preprod'
df_preview_merge['network'] = 'preview'

In [73]:
# df_merged = pd.concat([df_mainnet_merge, df_preprod_merge, df_preview_merge])
df_merged = pd.concat([df_preprod_merge, df_preview_merge])

In [74]:
df_merged

,year,month,num_stakeaddresses,num_contracts,num_stakeaddresses_runner,ada_transacted_mean,num_transactions_mean,ada_transacted_sum,num_transactions_sum,num_contracts_runner,network
0,2022,11,1,1,NaN,9999.584051,3.000000,9.999584e+03,3,0,preprod
1,2023,1,10,149,NaN,5925.309480,2.348993,8.828711e+05,350,0,preprod
2,2023,2,2,38,NaN,3262.007754,2.000000,1.239563e+05,76,0,preprod
3,2023,3,1,82,NaN,9565.109112,2.012195,7.843389e+05,165,0,preprod
4,2023,4,1,2,NaN,6398.745849,2.000000,1.279749e+04,4,0,preprod
5,2023,5,2,11,NaN,9591.847679,5.818182,1.055103e+05,64,0,preprod
6,2023,6,3,23,NaN,6735.362448,2.000000,1.549133e+05,46,0,preprod
7,2023,7,3,49,1.0,18022.615859,2.612245,8.831082e+05,128,1,preprod
8,2023,8,11,98,5.0,10040.368841,3.224490,9.839561e+05,316,9,preprod
9,2023,9,25,264,12.0,7651.776473,4.136364,2.020069e+06,1092,105,preprod


In [75]:
df_merged.to_sql(name='results_stakeaddress_monthly', con=engine_mainnet, if_exists='replace', schema='analysis', index=False)

24